In [24]:
from qonnx.core.modelwrapper import ModelWrapper
import onnxruntime as ort
from qonnx.transformation.create_generic_partitions import PartitionFromDict
from onnx import helper
from onnxruntime_extensions import get_library_path
from qonnx.util.basic import get_by_name
from onnx import checker


In [4]:
mfile = "estimate/Mul/Mul_0_out0/partition_1.onnx"
mfile_up = mfile.replace(".onnx","_ort.onnx")

model = ModelWrapper(mfile)

node = [n for n in model.graph.node if n.op_type == 'QuantAvgPool2d'][0]
attr = [n.attribute for n in model.graph.node if n.op_type == 'QuantAvgPool2d'][0]

for a in attr:
    if a.name == "stride":
        s = a.i
    elif a.name == "kernel":
        k = a.i
print(k,s)

update = helper.make_node(
    "AveragePool",
    inputs=[node.input[0]],
    outputs=[node.output[0]],
    kernel_shape=[k,k],
#     strides=[s,s],
)

for n in model.graph.node:
    if n.domain == "qonnx.custom_op.general":
        n.domain = "ai.onnx.contrib"
        
model.graph.node.remove(node)
# model.graph.node.append(update)

model.save(mfile_up)

so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())

sess = ort.InferenceSession(mfile_up, so)

7 1


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Load model from estimate/Mul/Mul_0_out0/partition_1_ort.onnx failed:Invalid model. Node input 'QuantAvgPool2d_0_out0' is not a graph input, initializer, or output of a previous node.

In [ ]:
model = ModelWrapper(mfile)
for n in model.graph.node:
    [print(n) for na in n.attribute if na.name == 'out_scale']

In [23]:
from qonnx.transformation.remove import remove_node_and_rewire
# from qonnx.util.cleanup import cleanup_model
model = ModelWrapper(mfile)
node = [n for n in model.graph.node if n.op_type == 'QuantAvgPool2d'][0]

remove_node_and_rewire(model,node)
# pre.output[0] 
# pre
# rm = []
# for i in range(node,len(model.graph.node)):
#     rm.append(model.graph.node[i])
# for n in rm:
#     model.graph.node.remove(n)
    
# new_out = model.get_tensor_valueinfo(pre.output[0])
# model.graph.output.remove(model.graph.output[0])
# model.graph.output.append(new_out)
for n in model.graph.node:
    if n.domain == "qonnx.custom_op.general":
        n.domain = "ai.onnx.contrib"
        
model.save(mfile_up)
so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())
checker.check_model(model.model,so)

# sess = ort.InferenceSession(mfile_up, so)



ValidationError: No opset import for domain 'ai.onnx.contrib'

==> Context: Bad node spec for node. Name: MultiThreshold_0 OpType: MultiThreshold

In [26]:
import onnxruntime as ort
import numpy as np
from onnxruntime_extensions import get_library_path
from qonnx.core.modelwrapper import ModelWrapper

mfile = "avgpool_test.onnx"
mfile_up = mfile.replace('.onnx','_ort.onnx')
model = ModelWrapper(mfile)

so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())
for n in model.graph.node:
    if n.domain == "qonnx.custom_op.general":
        n.domain = "ai.onnx.contrib"
    if n.op_type == "QuantAvgPool2d":
        n.domain = ''
model.save(mfile_up)

sess = ort.InferenceSession(mfile_up, so)
# sess

In [78]:
from onnx import helper, TensorProto, save

avgpool_attributes =  {"kernel_shape": [3, 3]}
avgpool_node = helper.make_node(
    "AveragePool", ["global_in"], ["global_out"], name="avgpool_node", **avgpool_attributes
)
inp_shape = [1, 3, 24, 40]
out_shape = [1, 3, 22, 38]
input_tensor = helper.make_tensor_value_info("global_in", TensorProto.FLOAT, inp_shape)
output_tensor = helper.make_tensor_value_info("global_out", TensorProto.FLOAT, out_shape)

graph = helper.make_graph(
    [avgpool_node],
    "AveragePool_test_model",
    [input_tensor],
    [output_tensor],
)
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 12)])
model.ir_version = 7  # use stable onnx ir version
save(model, "avgpool_test.onnx")
# avgpool_node